<a href="https://colab.research.google.com/github/Andrea-1704/Pytorch_Geometric_tutorial/blob/main/train_model_baseline_f1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Obiettivo
Il dataset su cui stiamo lavorando presenta tantissime tabelle (9). Domanda: sono tutte ugualmente importanti? qualcuna è "inutile"? capire la risposta a queste domande è il fine di questo notebook. L'idea è quella di costruire in modo compatto un loop in cui rimuoviamo una tabella prima della costruzione del grafo e valutare la differenza in termini di performance.


# Libraries to install

In [33]:
# !pip install torch==2.6.0+cu118 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# !pip install pyg-lib -f https://data.pyg.org/whl/torch-2.6.0+cu118.html
# !pip install torch-scatter -f https://data.pyg.org/whl/torch-2.6.0+cu118.html
# !pip install torch-sparse -f https://data.pyg.org/whl/torch-2.6.0+cu118.html
# !pip install torch-cluster -f https://data.pyg.org/whl/torch-2.6.0+cu118.html
# !pip install torch-spline-conv -f https://data.pyg.org/whl/torch-2.6.0+cu118.html
# !pip install torch-geometric==2.6.0 -f https://data.pyg.org/whl/torch-2.6.0+cu118.html

# !pip install pytorch_frame[full]==1.2.2
# !pip install relbench[full]==1.0.0

# Import

In [34]:
import os
import torch
import relbench
import numpy as np
from torch.nn import BCEWithLogitsLoss, L1Loss
from relbench.datasets import get_dataset
from relbench.tasks import get_task
import math
from tqdm import tqdm
import torch_geometric
import torch_frame
from torch_geometric.seed import seed_everything
from relbench.modeling.utils import get_stype_proposal
from collections import defaultdict
import requests
from io import StringIO
from torch_frame.config.text_embedder import TextEmbedderConfig
from relbench.modeling.graph import make_pkey_fkey_graph
from torch.nn import BCEWithLogitsLoss
import copy
from typing import Any, Dict, List
from torch import Tensor
from torch.nn import Embedding, ModuleDict
from torch_frame.data.stats import StatType
from torch_geometric.data import HeteroData
from torch_geometric.nn import MLP
from torch_geometric.typing import NodeType
from relbench.modeling.nn import HeteroEncoder, HeteroGraphSAGE, HeteroTemporalEncoder
from relbench.modeling.graph import get_node_train_table_input, make_pkey_fkey_graph
from torch_geometric.loader import NeighborLoader
import pyg_lib
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

#Utility functions

In [35]:
@torch.no_grad()
def alignment_check(loader: NeighborLoader, expected_node_ids: torch.Tensor):
    node_id_list = []

    for batch in loader:
        batch = batch.to(device)

        node_id_list.append(batch[task.entity_table].n_id.cpu())

    actual_node_ids = torch.cat(node_id_list, dim=0)

    assert len(actual_node_ids) == len(expected_node_ids), "Mismatch nella lunghezza"

    if not torch.equal(actual_node_ids, expected_node_ids):
        raise ValueError("Ordine dei nodi predetti diverso da val_table!")

    return

In [36]:
def evaluate_performance(pred: np.ndarray, target_table, metrics) -> dict:
    """Custom evaluation function to replace task.evaluate."""
    target = target_table.df[task.target_col].to_numpy()

    if len(pred) != len(target):
        raise ValueError(
            f"The length of pred and target must be the same (got "
            f"{len(pred)} and {len(target)}, respectively)."
        )

    results = {}
    for metric_fn in metrics:
        if metric_fn.__name__ == "rmse":
            results["rmse"] = np.sqrt(np.mean((target - pred)**2))
        else:
            results[metric_fn.__name__] = metric_fn(target, pred)

    return results

In [37]:
import numpy as np

def evaluate_on_train_during_training() -> float:
    model.eval()
    pred_list, target_list = [], []

    for batch in loader_dict["train"]:
        batch = batch.to(device)
        pred = model(batch, task.entity_table)
        pred = pred.view(-1) if pred.size(1) == 1 else pred
        pred_list.append(pred.detach().cpu())
        target_list.append(batch[task.entity_table].y.detach().cpu())

    pred_all = torch.cat(pred_list, dim=0).numpy()
    target_all = torch.cat(target_list, dim=0).numpy()

    mae = np.mean(np.abs(pred_all - target_all))
    return mae


In [38]:
def rmse(true, pred):
    """Calculate the Root Mean Squared Error (RMSE)."""
    return np.sqrt(np.mean((true - pred)**2))

In [39]:
@torch.no_grad()
def evaluate_on_full_train(model, loader) -> float:
    model.eval()
    pred_list, target_list = [], []

    for batch in loader:
        batch = batch.to(device)
        pred = model(batch, task.entity_table)
        pred = pred.view(-1) if pred.size(1) == 1 else pred
        pred_list.append(pred.cpu())
        target_list.append(batch[task.entity_table].y.cpu())

    pred_all = torch.cat(pred_list, dim=0).numpy()
    target_all = torch.cat(target_list, dim=0).numpy()

    mae = np.mean(np.abs(pred_all - target_all))
    return mae


In [40]:
class EarlyStopping:
    def __init__(self, patience=10, delta=0, verbose=False, path='checkpoint.pt'):
        """
        Args:
            patience (int): Quanto aspettare senza miglioramenti prima di fermare.
            delta (float): Miglioramento minimo richiesto per considerare un miglioramento.
            verbose (bool): Se stampare informazioni.
            path (str): Dove salvare il modello migliore.
        """
        self.patience = patience
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float('inf')
        self.delta = delta
        self.verbose = verbose
        self.path = path

    def __call__(self, val_loss, model):

        score = -val_loss  # Perché vogliamo MINIMIZZARE la loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)

        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} / {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True

        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Salva il modello migliore'''
        if self.verbose:
            print(f'Validation loss migliorata ({self.val_loss_min:.6f} --> {val_loss:.6f}). Salvo modello...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss


# Dataset and task creation

In [59]:
dataset = get_dataset("rel-f1", download=True)
task = get_task("rel-f1", "driver-position", download=True)

train_table = task.get_table("train") #date  driverId  qualifying
val_table = task.get_table("val") #date  driverId  qualifying
test_table = task.get_table("test") # date  driverId

out_channels = 1
loss_fn = L1Loss()
# this is the mae loss and is used when have regressions tasks.
tune_metric = "mae"
higher_is_better = False

seed_everything(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
root_dir = "./data"

db = dataset.get_db() #get all tables
col_to_stype_dict = get_stype_proposal(db)
entity_table = task.entity_table
#this is used to get the stype of the columns

cuda


In [42]:
list(db.table_dict.keys())

['circuits',
 'results',
 'qualifying',
 'standings',
 'constructor_standings',
 'constructors',
 'constructor_results',
 'races',
 'drivers']

# Embedder

In [43]:
class LightweightGloveEmbedder:
    def __init__(self, device=None):
        self.device = device
        self.embeddings = defaultdict(lambda: np.zeros(300))
        self._load_embeddings()

    def _load_embeddings(self):
      try:
          path = "glove.6B.300d.txt"
          with open(path, encoding="utf-8") as f:
              for line in f:
                  parts = line.strip().split()
                  word = parts[0]
                  vector = np.array(parts[1:], dtype=np.float32)
                  self.embeddings[word] = vector
          #print(f"Loaded {len(self.embeddings)} GloVe embeddings.")
      except Exception as e:
          print(f"Failed to load GloVe: {e}")

    def __call__(self, sentences):
        results = []
        for text in sentences:
            words = text.lower().split()
            vectors = [self.embeddings[w] for w in words if w in self.embeddings]
            if vectors:
                avg_vector = np.mean(vectors, axis=0)
            else:
                #print("non trovato")
                #print(f"Numero parole in embedding: {len(self.embeddings)}")

                avg_vector = np.zeros(300)
            results.append(avg_vector)

        tensor = torch.tensor(np.array(results), dtype=torch.float32)
        return tensor.to(self.device) if self.device else tensor

In [44]:
text_embedder_cfg = TextEmbedderConfig(
    text_embedder=LightweightGloveEmbedder(device=device), batch_size=256
)

# data, col_stats_dict = make_pkey_fkey_graph(
#     db,
#     col_to_stype_dict=col_to_stype_dict,
#     text_embedder_cfg=text_embedder_cfg,
#     cache_dir=os.path.join(
#         root_dir, f"rel-f1_materialized_cache"
#     ),
# )

# Graph Loader

In [45]:
# qui i parametri di train_table, val_table, test_table, task e data sono
#parametri globali

def loader_dict_fn(batch_size, num_neighbours, data):
    loader_dict = {}

    for split, table in [
        ("train", train_table),
        ("val", val_table),
        ("test", test_table),
    ]:
        table_input = get_node_train_table_input(
            table=table,
            task=task,
        )

        loader_dict[split] = NeighborLoader(
            data,
            num_neighbors=[num_neighbours for _ in range(2)],
            time_attr="time",
            input_nodes=table_input.nodes,
            input_time=table_input.time,
            transform=table_input.transform,
            batch_size=batch_size,
            temporal_strategy="uniform",
            shuffle=split == "train",
            num_workers=0,
            persistent_workers=False,
        )

    return loader_dict


# Model

In [46]:
class Model(torch.nn.Module):

    def __init__(
        self,
        data: HeteroData,
        col_stats_dict: Dict[str, Dict[str, Dict[StatType, Any]]],
        num_layers: int,
        channels: int,
        out_channels: int,
        aggr: str,
        norm: str,
        shallow_list: List[NodeType] = [],
        id_awareness: bool = False,
    ):
        super().__init__()

        self.encoder = HeteroEncoder(
            channels=channels,
            node_to_col_names_dict={
                node_type: data[node_type].tf.col_names_dict
                for node_type in data.node_types
            },
            node_to_col_stats=col_stats_dict,
        )

        self.temporal_encoder = HeteroTemporalEncoder(
            node_types=[
                node_type for node_type in data.node_types if "time" in data[node_type]
            ],
            channels=channels,
        )

        self.gnn = HeteroGraphSAGE(
            node_types=data.node_types,
            edge_types=data.edge_types,
            channels=channels,
            aggr=aggr,
            num_layers=num_layers,
        )
        self.head = MLP(
            channels,
            out_channels=out_channels,
            norm=norm,
            num_layers=1, ###################################################
        )
        self.embedding_dict = ModuleDict(
            {
                node: Embedding(data.num_nodes_dict[node], channels)
                for node in shallow_list
            }
        )

        self.id_awareness_emb = None
        if id_awareness:
            self.id_awareness_emb = torch.nn.Embedding(1, channels)
        self.reset_parameters()

    def reset_parameters(self):
        self.encoder.reset_parameters()
        self.temporal_encoder.reset_parameters()
        self.gnn.reset_parameters()
        self.head.reset_parameters()
        for embedding in self.embedding_dict.values():
            torch.nn.init.normal_(embedding.weight, std=0.1)
        if self.id_awareness_emb is not None:
            self.id_awareness_emb.reset_parameters()

    def forward(
        self,
        batch: HeteroData,
        entity_table: NodeType,
    ) -> Tensor:
        seed_time = batch[entity_table].seed_time
        x_dict = self.encoder(batch.tf_dict)

        rel_time_dict = self.temporal_encoder(
            seed_time, batch.time_dict, batch.batch_dict
        )

        for node_type, rel_time in rel_time_dict.items():
            x_dict[node_type] = x_dict[node_type] + rel_time

        for node_type, embedding in self.embedding_dict.items():
            x_dict[node_type] = x_dict[node_type] + embedding(batch[node_type].n_id)

        x_dict = self.gnn(
            x_dict,
            batch.edge_index_dict,
            batch.num_sampled_nodes_dict,
            batch.num_sampled_edges_dict,
        )

        return self.head(x_dict[entity_table][: seed_time.size(0)])

    def forward_dst_readout(
        self,
        batch: HeteroData,
        entity_table: NodeType,
        dst_table: NodeType,
    ) -> Tensor:
        if self.id_awareness_emb is None:
            raise RuntimeError(
                "id_awareness must be set True to use forward_dst_readout"
            )
        seed_time = batch[entity_table].seed_time
        x_dict = self.encoder(batch.tf_dict)
        x_dict[entity_table][: seed_time.size(0)] += self.id_awareness_emb.weight

        rel_time_dict = self.temporal_encoder(
            seed_time, batch.time_dict, batch.batch_dict
        )

        for node_type, rel_time in rel_time_dict.items():
            x_dict[node_type] = x_dict[node_type] + rel_time

        for node_type, embedding in self.embedding_dict.items():
            x_dict[node_type] = x_dict[node_type] + embedding(batch[node_type].n_id)

        x_dict = self.gnn(
            x_dict,
            batch.edge_index_dict,
        )

        return self.head(x_dict[dst_table])

# Training functions

Ora necessito di modificare la funzione di train per prendere anche il valore del loader_dict: utile per tuning dei parametri (vedi il codice della funzione di tuning).

In [47]:
def train(model, optimizer, loader_dict) -> float:
    model.train()

    loss_accum = count_accum = 0
    for batch in tqdm(loader_dict["train"]):
        batch = batch.to(device)

        optimizer.zero_grad()
        pred = model(
            batch,
            task.entity_table,
        )
        pred = pred.view(-1) if pred.size(1) == 1 else pred

        loss = loss_fn(pred.float(), batch[entity_table].y.float())
        loss.backward()
        optimizer.step()

        loss_accum += loss.detach().item() * pred.size(0)
        count_accum += pred.size(0)

    return loss_accum / count_accum


@torch.no_grad()
def test(model, loader: NeighborLoader) -> np.ndarray:
    model.eval()

    pred_list = []
    for batch in loader:
        batch = batch.to(device)
        pred = model(
            batch,
            task.entity_table,
        )
        pred = pred.view(-1) if pred.size(1) == 1 else pred
        pred_list.append(pred.detach().cpu())
    return torch.cat(pred_list, dim=0).numpy()

In [48]:
print(task.target_col)

position


# Ablation test on tables

## Preliminary analysis

Codice per rimuovere una tabella specifica dal db

Ma, attenzione, questo non basta perché non basta rimuovere semplicemente la tabella, ma dobbiamo anche rimuovere ogni riferimento di chiave esterno che si riferiscono a quella tabella. Questo lo facciamo sott.

In [53]:
import copy
from relbench.modeling.graph import make_pkey_fkey_graph

def build_graph_excluding_table(dataset, table_to_remove, root_dir, text_embedder_cfg):
    # 1. Deepcopy del db originale
    original_db = dataset.get_db()
    db = copy.deepcopy(original_db)

    # 2. Elimina la tabella dal table_dict
    if table_to_remove in db.table_dict:
        del db.table_dict[table_to_remove]

    # 3. Rimuovi tutti i fkey che puntano alla tabella rimossa
    for table_name, table in db.table_dict.items():
        table.fkey_col_to_pkey_table = {
            col: tgt for col, tgt in table.fkey_col_to_pkey_table.items()
            if tgt != table_to_remove
        }

    # 4. Costruisci col_to_stype_dict filtrato
    full_stype = get_stype_proposal(original_db)
    filtered_stype = {
        tab: stype for tab, stype in full_stype.items()
        if tab != table_to_remove
    }

    # 5. Costruisci il grafo
    data, col_stats_dict = make_pkey_fkey_graph(
        db,
        col_to_stype_dict=filtered_stype,
        text_embedder_cfg=text_embedder_cfg,
        cache_dir=os.path.join(root_dir, f"ablation_cache_{table_to_remove}")
    )

    return data, col_stats_dict


In [54]:
all_tables = list(db.table_dict.keys())

In [55]:
print(all_tables)

['circuits', 'results', 'qualifying', 'standings', 'constructor_standings', 'constructors', 'constructor_results', 'races', 'drivers']


In [62]:
all_tables = list(db.table_dict.keys())
ablation_results = []

for table_to_remove in all_tables:
    if table_to_remove == task.entity_table:
      print(f"Skipping ablation on target table: {table_to_remove}")
      continue

    print(f"Testing without table: {table_to_remove}")

    # Copia del DB con una tabella rimossa
    import copy
    #dataset_copy = dataset.deepcopy(dataset)

    #db_copy.pop(table_to_remove)
    #db_filtered = build_filtered_db(db_copy, table_to_remove)

    # Ricostruisci il grafo

    data, col_stats_dict = build_graph_excluding_table(
        dataset, table_to_remove=table_to_remove,
        root_dir=root_dir,
        text_embedder_cfg=text_embedder_cfg
    )

    loader_dict = loader_dict_fn(batch_size=512, num_neighbours=256, data=data)

    model = Model(
        data=data,
        col_stats_dict=col_stats_dict,
        num_layers=2,
        channels=128,
        out_channels=1,
        aggr="max",
        norm="batch_norm",
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=0)

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode="min",
        factor=0.5,
        patience=10,
        verbose=True
    )

    for epoch in range(15):  # Poche epoche per test rapido
        train(model, optimizer, loader_dict)

    val_pred = test(model, loader_dict["val"])
    val_mae = evaluate_performance(val_pred, val_table, task.metrics)["mae"]

    ablation_results.append((table_to_remove, val_mae))


Testing without table: circuits


/usr/local/lib/python3.11/dist-packages/torch_frame/utils/io.py:113: UserWarning: Weights only load failed. Please file an issue to make `torch.load(weights_only=True)` compatible in your case. Please use `torch.serialization.add_safe_globals([scalar])` to allowlist this global.
  warnings.warn(f"{warn_msg} Please use "
/usr/local/lib/python3.11/dist-packages/torch_frame/utils/io.py:113: UserWarning: Weights only load failed. Please file an issue to make `torch.load(weights_only=True)` compatible in your case. Please use `torch.serialization.add_safe_globals([scalar])` to allowlist this global.
  warnings.warn(f"{warn_msg} Please use "
/usr/local/lib/python3.11/dist-packages/torch_frame/utils/io.py:113: UserWarning: Weights only load failed. Please file an issue to make `torch.load(weights_only=True)` compatible in your case. Please use `torch.serialization.add_safe_globals([scalar])` to allowlist this global.
  warnings.warn(f"{warn_msg} Please use "
/usr/local/lib/python3.11/dist-pac

Testing without table: results


/usr/local/lib/python3.11/dist-packages/torch_frame/utils/io.py:113: UserWarning: Weights only load failed. Please file an issue to make `torch.load(weights_only=True)` compatible in your case. Please use `torch.serialization.add_safe_globals([scalar])` to allowlist this global.
  warnings.warn(f"{warn_msg} Please use "
/usr/local/lib/python3.11/dist-packages/torch_frame/utils/io.py:113: UserWarning: Weights only load failed. Please file an issue to make `torch.load(weights_only=True)` compatible in your case. Please use `torch.serialization.add_safe_globals([scalar])` to allowlist this global.
  warnings.warn(f"{warn_msg} Please use "
/usr/local/lib/python3.11/dist-packages/torch_frame/utils/io.py:113: UserWarning: Weights only load failed. Please file an issue to make `torch.load(weights_only=True)` compatible in your case. Please use `torch.serialization.add_safe_globals([scalar])` to allowlist this global.
  warnings.warn(f"{warn_msg} Please use "
/usr/local/lib/python3.11/dist-pac

Testing without table: qualifying


...

Testing without table: constructor_results


/usr/local/lib/python3.11/dist-packages/torch_frame/utils/io.py:113: UserWarning: Weights only load failed. Please file an issue to make `torch.load(weights_only=True)` compatible in your case. Please use `torch.serialization.add_safe_globals([scalar])` to allowlist this global.
  warnings.warn(f"{warn_msg} Please use "
/usr/local/lib/python3.11/dist-packages/torch_frame/utils/io.py:113: UserWarning: Weights only load failed. Please file an issue to make `torch.load(weights_only=True)` compatible in your case. Please use `torch.serialization.add_safe_globals([scalar])` to allowlist this global.
  warnings.warn(f"{warn_msg} Please use "
/usr/local/lib/python3.11/dist-packages/torch_frame/utils/io.py:113: UserWarning: Weights only load failed. Please file an issue to make `torch.load(weights_only=True)` compatible in your case. Please use `torch.serialization.add_safe_globals([scalar])` to allowlist this global.
  warnings.warn(f"{warn_msg} Please use "
/usr/local/lib/python3.11/dist-pac

Testing without table: races


/usr/local/lib/python3.11/dist-packages/torch_frame/utils/io.py:113: UserWarning: Weights only load failed. Please file an issue to make `torch.load(weights_only=True)` compatible in your case. Please use `torch.serialization.add_safe_globals([scalar])` to allowlist this global.
  warnings.warn(f"{warn_msg} Please use "
/usr/local/lib/python3.11/dist-packages/torch_frame/utils/io.py:113: UserWarning: Weights only load failed. Please file an issue to make `torch.load(weights_only=True)` compatible in your case. Please use `torch.serialization.add_safe_globals([scalar])` to allowlist this global.
  warnings.warn(f"{warn_msg} Please use "
/usr/local/lib/python3.11/dist-packages/torch_frame/utils/io.py:113: UserWarning: Weights only load failed. Please file an issue to make `torch.load(weights_only=True)` compatible in your case. Please use `torch.serialization.add_safe_globals([scalar])` to allowlist this global.
  warnings.warn(f"{warn_msg} Please use "
/usr/local/lib/python3.11/dist-pac

Skipping ablation on target table: drivers


In [63]:
ablation_results.sort(key=lambda x: x[1])  # ordina per MAE crescente
for table, mae in ablation_results:
    print(f"{table:20s} → val MAE: {mae:.4f}")


circuits             → val MAE: 3.8530
constructor_results  → val MAE: 3.8537
qualifying           → val MAE: 3.8641
standings            → val MAE: 3.8670
results              → val MAE: 3.8696
races                → val MAE: 3.8744
constructors         → val MAE: 3.8816
constructor_standings → val MAE: 3.8895


Adesso vediamo il modello completo che risultati ottiene con epoche pari a 15

In [68]:
all_tables = list(db.table_dict.keys())
ablation_results = []
print(all_tables)
data, col_stats_dict = build_graph_excluding_table(
        dataset, table_to_remove=None,
        root_dir=root_dir,
        text_embedder_cfg=text_embedder_cfg
    )

loader_dict = loader_dict_fn(batch_size=512, num_neighbours=256, data=data)

model = Model(
        data=data,
        col_stats_dict=col_stats_dict,
        num_layers=2,
        channels=128,
        out_channels=1,
        aggr="max",
        norm="batch_norm",
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=0)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode="min",
        factor=0.5,
        patience=10,
        verbose=True
)

for epoch in range(15):  # Poche epoche per test rapido
        train(model, optimizer, loader_dict)

val_pred = test(model, loader_dict["val"])
val_mae = evaluate_performance(val_pred, val_table, task.metrics)["mae"]

['circuits', 'results', 'qualifying', 'standings', 'constructor_standings', 'constructors', 'constructor_results', 'races', 'drivers']


/usr/local/lib/python3.11/dist-packages/torch_frame/utils/io.py:113: UserWarning: Weights only load failed. Please file an issue to make `torch.load(weights_only=True)` compatible in your case. Please use `torch.serialization.add_safe_globals([scalar])` to allowlist this global.
  warnings.warn(f"{warn_msg} Please use "
/usr/local/lib/python3.11/dist-packages/torch_frame/utils/io.py:113: UserWarning: Weights only load failed. Please file an issue to make `torch.load(weights_only=True)` compatible in your case. Please use `torch.serialization.add_safe_globals([scalar])` to allowlist this global.
  warnings.warn(f"{warn_msg} Please use "
/usr/local/lib/python3.11/dist-packages/torch_frame/utils/io.py:113: UserWarning: Weights only load failed. Please file an issue to make `torch.load(weights_only=True)` compatible in your case. Please use `torch.serialization.add_safe_globals([scalar])` to allowlist this global.
  warnings.warn(f"{warn_msg} Please use "
/usr/local/lib/python3.11/dist-pac

In [69]:
print(val_mae)

3.8667647586636487


| Tabella rimossa         | val MAE | Δ rispetto a baseline | Interpretazione                           |
| ----------------------- | ------- | --------------------- | ----------------------------------------- |
| `circuits`              | 3.8530  | **−0.0208**           | 🔥 Migliora → possibile tabella rumore    |
| `constructor_results`   | 3.8537  | −0.0201               | 🔥 Migliora leggermente                   |
| `qualifying`            | 3.8641  | −0.0097               | ↘️ Lieve miglioramento                    |
| `standings`             | 3.8670  | −0.0068               | ↘️ Lieve miglioramento                    |
| `results`               | 3.8696  | −0.0042               | 🔸 Quasi neutro                           |
| `races`                 | 3.8744  | +0.0006               | 🔸 Identico                               |
| `constructors`          | 3.8816  | +0.0078               | ↗️ Lieve peggioramento                    |
| `constructor_standings` | 3.8895  | +0.0157               | 🔼 Peggiora → tabella probabilmente utile |


# Removing circuits

Proviamo a rimuovere le tabelle: circuits

In [70]:
print(f"Testing without table: circuits")
table_to_remove = "circuits"

data, col_stats_dict = build_graph_excluding_table(
        dataset, table_to_remove=table_to_remove,
        root_dir=root_dir,
        text_embedder_cfg=text_embedder_cfg
)

loader_dict = loader_dict_fn(batch_size=512, num_neighbours=256, data=data)

model = Model(
        data=data,
        col_stats_dict=col_stats_dict,
        num_layers=2,
        channels=128,
        out_channels=1,
        aggr="max",
        norm="batch_norm",
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=0)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode="min",
        factor=0.5,
        patience=10,
        verbose=True
)

early_stopping = EarlyStopping(
    patience=30,
    delta=0.0,
    verbose=True,
    path="best_basic_model.pt"
)

epochs = 600
state_dict = None
test_table = task.get_table("test", mask_input_cols=False)
best_val_metric = -math.inf if higher_is_better else math.inf
best_test_metric = -math.inf if higher_is_better else math.inf
for epoch in range(1, epochs + 1):
    train_loss = train(model, optimizer, loader_dict=loader_dict)

    train_pred = test(model, loader_dict["train"])
    train_metrics = evaluate_performance(train_pred, train_table, task.metrics)
    train_mae_preciso = evaluate_on_full_train(model, loader_dict["train"])

    val_pred = test(model, loader_dict["val"])
    val_metrics = evaluate_performance(val_pred, val_table, task.metrics)

    test_pred = test(model, loader_dict["test"])
    test_metrics = evaluate_performance(test_pred, test_table, task.metrics)

    scheduler.step(val_metrics[tune_metric])

    if (higher_is_better and val_metrics[tune_metric] > best_val_metric) or (
            not higher_is_better and val_metrics[tune_metric] < best_val_metric
    ):
        best_val_metric = val_metrics[tune_metric]
        state_dict = copy.deepcopy(model.state_dict())

    #test:
    if (higher_is_better and test_metrics[tune_metric] > best_test_metric) or (
            not higher_is_better and test_metrics[tune_metric] < best_test_metric
    ):
        best_test_metric = test_metrics[tune_metric]
        state_dict_test = copy.deepcopy(model.state_dict())

    current_lr = optimizer.param_groups[0]["lr"]
    print(f"Epoch: {epoch:02d}, Train mae: {train_mae_preciso:.2f}, Validation MAE: {val_metrics[tune_metric]:.2f}, Test MAE: {test_metrics[tune_metric]:.2f}, LR: {current_lr:.6f}")

    early_stopping(val_metrics[tune_metric], model)

    if early_stopping.early_stop:
        print(f"Early stopping triggered at epoch {epoch}")
        break
print(f"best validation results: {best_val_metric}")
print(f"best test results: {best_test_metric}")

Testing without table: circuits


/usr/local/lib/python3.11/dist-packages/torch_frame/utils/io.py:113: UserWarning: Weights only load failed. Please file an issue to make `torch.load(weights_only=True)` compatible in your case. Please use `torch.serialization.add_safe_globals([scalar])` to allowlist this global.
  warnings.warn(f"{warn_msg} Please use "
/usr/local/lib/python3.11/dist-packages/torch_frame/utils/io.py:113: UserWarning: Weights only load failed. Please file an issue to make `torch.load(weights_only=True)` compatible in your case. Please use `torch.serialization.add_safe_globals([scalar])` to allowlist this global.
  warnings.warn(f"{warn_msg} Please use "
/usr/local/lib/python3.11/dist-packages/torch_frame/utils/io.py:113: UserWarning: Weights only load failed. Please file an issue to make `torch.load(weights_only=True)` compatible in your case. Please use `torch.serialization.add_safe_globals([scalar])` to allowlist this global.
  warnings.warn(f"{warn_msg} Please use "
/usr/local/lib/python3.11/dist-pac

Epoch: 01, Train mae: 9.86, Validation MAE: 7.07, Test MAE: 7.90, LR: 0.000500
Validation loss migliorata (inf --> 7.073873). Salvo modello...


100%|██████████| 15/15 [00:02<00:00,  5.55it/s]


Epoch: 02, Train mae: 9.34, Validation MAE: 6.57, Test MAE: 7.40, LR: 0.000500
Validation loss migliorata (7.073873 --> 6.565605). Salvo modello...


...

Epoch: 64, Train mae: 2.76, Validation MAE: 3.21, Test MAE: 4.80, LR: 0.000125
EarlyStopping counter: 28 / 30


100%|██████████| 15/15 [00:02<00:00,  5.11it/s]


Epoch: 65, Train mae: 2.76, Validation MAE: 3.36, Test MAE: 4.84, LR: 0.000125
EarlyStopping counter: 29 / 30


100%|██████████| 15/15 [00:02<00:00,  5.52it/s]


Epoch: 66, Train mae: 2.59, Validation MAE: 3.28, Test MAE: 4.74, LR: 0.000125
EarlyStopping counter: 30 / 30
Early stopping triggered at epoch 66
best validation results: 2.7886699412135014
best test results: 3.8109309199818395


Proviamo adesso a rimuovere sia circuits che constructor_results:

# Multiple tables removal

Progetto una funzione che effettua la rimozione per lista di tables:

In [ ]:
def build_graph_excluding_tables(dataset, tables_to_remove, root_dir, text_embedder_cfg):
    import copy
    from relbench.modeling.graph import make_pkey_fkey_graph

    # 1. Deepcopy del db originale
    original_db = dataset.get_db()
    db = copy.deepcopy(original_db)

    # 2. Rimuovi tutte le tabelle
    for table_to_remove in tables_to_remove:
        if table_to_remove in db.table_dict:
            del db.table_dict[table_to_remove]

    # 3. Rimuovi fkey che puntano a tabelle escluse
    for table in db.table_dict.values():
        table.fkey_col_to_pkey_table = {
            col: tgt for col, tgt in table.fkey_col_to_pkey_table.items()
            if tgt not in tables_to_remove
        }
        # Rimuoviamo anche le colonne fkey nei df
        table.df = table.df.drop(
            columns=[
                col for col, tgt in table.fkey_col_to_pkey_table.items()
                if tgt not in db.table_dict
            ],
            errors="ignore"
        )

    # 4. Filtra anche lo stype
    full_stype = get_stype_proposal(original_db)
    filtered_stype = {
        tab: stype for tab, stype in full_stype.items()
        if tab not in tables_to_remove
    }

    # 5. Costruzione del grafo UNA sola volta
    cache_name = "_".join(sorted(tables_to_remove))
    data, col_stats_dict = make_pkey_fkey_graph(
        db,
        col_to_stype_dict=filtered_stype,
        text_embedder_cfg=text_embedder_cfg,
        cache_dir=os.path.join(root_dir, f"ablation_cache_{cache_name}")
    )

    all_tables = list(db.table_dict.keys())
    print(f"Tabelle rimanenti nel grafo: {all_tables}")

    return data, col_stats_dict


## circuits and constructor_results

In [93]:
print(f"Testing without table: circuits and constructor_result")
tables_to_remove = ["circuits","constructor_results"]

data, col_stats_dict = build_graph_excluding_tables(
        dataset, tables_to_remove=tables_to_remove,
        root_dir=root_dir,
        text_embedder_cfg=text_embedder_cfg
)

loader_dict = loader_dict_fn(batch_size=512, num_neighbours=256, data=data)

model = Model(
        data=data,
        col_stats_dict=col_stats_dict,
        num_layers=2,
        channels=128,
        out_channels=1,
        aggr="max",
        norm="batch_norm",
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=0)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode="min",
        factor=0.5,
        patience=10,
        verbose=True
)

early_stopping = EarlyStopping(
    patience=30,
    delta=0.0,
    verbose=True,
    path="best_basic_model.pt"
)

epochs = 600
state_dict = None
test_table = task.get_table("test", mask_input_cols=False)
best_val_metric = -math.inf if higher_is_better else math.inf
best_test_metric = -math.inf if higher_is_better else math.inf
for epoch in range(1, epochs + 1):
    train_loss = train(model, optimizer, loader_dict=loader_dict)

    train_pred = test(model, loader_dict["train"])
    train_metrics = evaluate_performance(train_pred, train_table, task.metrics)
    train_mae_preciso = evaluate_on_full_train(model, loader_dict["train"])

    val_pred = test(model, loader_dict["val"])
    val_metrics = evaluate_performance(val_pred, val_table, task.metrics)

    test_pred = test(model, loader_dict["test"])
    test_metrics = evaluate_performance(test_pred, test_table, task.metrics)

    scheduler.step(val_metrics[tune_metric])

    if (higher_is_better and val_metrics[tune_metric] > best_val_metric) or (
            not higher_is_better and val_metrics[tune_metric] < best_val_metric
    ):
        best_val_metric = val_metrics[tune_metric]
        state_dict = copy.deepcopy(model.state_dict())

    #test:
    if (higher_is_better and test_metrics[tune_metric] > best_test_metric) or (
            not higher_is_better and test_metrics[tune_metric] < best_test_metric
    ):
        best_test_metric = test_metrics[tune_metric]
        state_dict_test = copy.deepcopy(model.state_dict())

    current_lr = optimizer.param_groups[0]["lr"]
    print(f"Epoch: {epoch:02d}, Train mae: {train_mae_preciso:.2f}, Validation MAE: {val_metrics[tune_metric]:.2f}, Test MAE: {test_metrics[tune_metric]:.2f}, LR: {current_lr:.6f}")

    early_stopping(val_metrics[tune_metric], model)

    if early_stopping.early_stop:
        print(f"Early stopping triggered at epoch {epoch}")
        break
print(f"best validation results: {best_val_metric}")
print(f"best test results: {best_test_metric}")

Testing without table: circuits and constructor_result
Tabelle rimanenti nel grafo: ['results', 'qualifying', 'standings', 'constructor_standings', 'constructors', 'races', 'drivers']


/usr/local/lib/python3.11/dist-packages/torch_frame/utils/io.py:113: UserWarning: Weights only load failed. Please file an issue to make `torch.load(weights_only=True)` compatible in your case. Please use `torch.serialization.add_safe_globals([scalar])` to allowlist this global.
  warnings.warn(f"{warn_msg} Please use "
/usr/local/lib/python3.11/dist-packages/torch_frame/utils/io.py:113: UserWarning: Weights only load failed. Please file an issue to make `torch.load(weights_only=True)` compatible in your case. Please use `torch.serialization.add_safe_globals([scalar])` to allowlist this global.
  warnings.warn(f"{warn_msg} Please use "
/usr/local/lib/python3.11/dist-packages/torch_frame/utils/io.py:113: UserWarning: Weights only load failed. Please file an issue to make `torch.load(weights_only=True)` compatible in your case. Please use `torch.serialization.add_safe_globals([scalar])` to allowlist this global.
  warnings.warn(f"{warn_msg} Please use "
/usr/local/lib/python3.11/dist-pac

Epoch: 01, Train mae: 10.33, Validation MAE: 7.51, Test MAE: 8.40, LR: 0.000500
Validation loss migliorata (inf --> 7.507688). Salvo modello...


100%|██████████| 15/15 [00:02<00:00,  5.02it/s]


Epoch: 02, Train mae: 9.87, Validation MAE: 7.06, Test MAE: 7.90, LR: 0.000500
Validation loss migliorata (7.507688 --> 7.064141). Salvo modello...


...

Epoch: 62, Train mae: 2.65, Validation MAE: 3.11, Test MAE: 4.92, LR: 0.000125
EarlyStopping counter: 28 / 30


100%|██████████| 15/15 [00:02<00:00,  5.30it/s]


Epoch: 63, Train mae: 2.64, Validation MAE: 3.01, Test MAE: 4.79, LR: 0.000125
EarlyStopping counter: 29 / 30


100%|██████████| 15/15 [00:02<00:00,  5.63it/s]


Epoch: 64, Train mae: 2.59, Validation MAE: 3.12, Test MAE: 4.82, LR: 0.000125
EarlyStopping counter: 30 / 30
Early stopping triggered at epoch 64
best validation results: 2.7831582932290666
best test results: 3.8213064945789808


Sembra che la rete benefici di queste rimozioni!

## Circuits, constructor_results, standings, qualifying

PROVIAMO A RIMUOVERE CIRCUITS, CONSTRUCTOR_RESULTS, STANDINGS E qualifying

In [90]:
print(f"Testing without table: circuits,constructor_results, standings, qualifying")
tables_to_remove = ["circuits","constructor_results", "standings", "qualifying"]

data, col_stats_dict = build_graph_excluding_tables(
        dataset, tables_to_remove=tables_to_remove,
        root_dir=root_dir,
        text_embedder_cfg=text_embedder_cfg
)

loader_dict = loader_dict_fn(batch_size=512, num_neighbours=256, data=data)

model = Model(
        data=data,
        col_stats_dict=col_stats_dict,
        num_layers=2,
        channels=128,
        out_channels=1,
        aggr="max",
        norm="batch_norm",
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=0)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode="min",
        factor=0.5,
        patience=10,
        verbose=True
)

early_stopping = EarlyStopping(
    patience=30,
    delta=0.0,
    verbose=True,
    path="best_basic_model.pt"
)

epochs = 600
state_dict = None
test_table = task.get_table("test", mask_input_cols=False)
best_val_metric = -math.inf if higher_is_better else math.inf
best_test_metric = -math.inf if higher_is_better else math.inf
for epoch in range(1, epochs + 1):
    train_loss = train(model, optimizer, loader_dict=loader_dict)

    train_pred = test(model, loader_dict["train"])
    train_metrics = evaluate_performance(train_pred, train_table, task.metrics)
    train_mae_preciso = evaluate_on_full_train(model, loader_dict["train"])

    val_pred = test(model, loader_dict["val"])
    val_metrics = evaluate_performance(val_pred, val_table, task.metrics)

    test_pred = test(model, loader_dict["test"])
    test_metrics = evaluate_performance(test_pred, test_table, task.metrics)

    scheduler.step(val_metrics[tune_metric])

    if (higher_is_better and val_metrics[tune_metric] > best_val_metric) or (
            not higher_is_better and val_metrics[tune_metric] < best_val_metric
    ):
        best_val_metric = val_metrics[tune_metric]
        state_dict = copy.deepcopy(model.state_dict())

    #test:
    if (higher_is_better and test_metrics[tune_metric] > best_test_metric) or (
            not higher_is_better and test_metrics[tune_metric] < best_test_metric
    ):
        best_test_metric = test_metrics[tune_metric]
        state_dict_test = copy.deepcopy(model.state_dict())

    current_lr = optimizer.param_groups[0]["lr"]
    print(f"Epoch: {epoch:02d}, Train mae: {train_mae_preciso:.2f}, Validation MAE: {val_metrics[tune_metric]:.2f}, Test MAE: {test_metrics[tune_metric]:.2f}, LR: {current_lr:.6f}")

    early_stopping(val_metrics[tune_metric], model)

    if early_stopping.early_stop:
        print(f"Early stopping triggered at epoch {epoch}")
        break
print(f"best validation results: {best_val_metric}")
print(f"best test results: {best_test_metric}")

Testing without table: circuits,constructor_results, standings, qualifying


/usr/local/lib/python3.11/dist-packages/torch_frame/utils/io.py:113: UserWarning: Weights only load failed. Please file an issue to make `torch.load(weights_only=True)` compatible in your case. Please use `torch.serialization.add_safe_globals([scalar])` to allowlist this global.
  warnings.warn(f"{warn_msg} Please use "
/usr/local/lib/python3.11/dist-packages/torch_frame/utils/io.py:113: UserWarning: Weights only load failed. Please file an issue to make `torch.load(weights_only=True)` compatible in your case. Please use `torch.serialization.add_safe_globals([scalar])` to allowlist this global.
  warnings.warn(f"{warn_msg} Please use "
/usr/local/lib/python3.11/dist-packages/torch_frame/utils/io.py:113: UserWarning: Weights only load failed. Please file an issue to make `torch.load(weights_only=True)` compatible in your case. Please use `torch.serialization.add_safe_globals([scalar])` to allowlist this global.
  warnings.warn(f"{warn_msg} Please use "
/usr/local/lib/python3.11/dist-pac

Tabelle rimanenti nel grafo: ['results', 'constructor_standings', 'constructors', 'races', 'drivers']


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
100%|██████████| 15/15 [00:01<00:00,  8.52it/s]


Epoch: 01, Train mae: 10.56, Validation MAE: 7.71, Test MAE: 8.71, LR: 0.000500
Validation loss migliorata (inf --> 7.707859). Salvo modello...


...

Epoch: 66, Train mae: 3.93, Validation MAE: 2.98, Test MAE: 4.32, LR: 0.000063
EarlyStopping counter: 28 / 30


100%|██████████| 15/15 [00:01<00:00,  8.32it/s]


Epoch: 67, Train mae: 3.90, Validation MAE: 2.96, Test MAE: 4.32, LR: 0.000063
EarlyStopping counter: 29 / 30


100%|██████████| 15/15 [00:01<00:00,  9.17it/s]


Epoch: 68, Train mae: 3.90, Validation MAE: 3.04, Test MAE: 4.25, LR: 0.000063
EarlyStopping counter: 30 / 30
Early stopping triggered at epoch 68
best validation results: 2.8463728568995084
best test results: 3.9194119374793868
